In [ ]:
%matplotlib inline

In [ ]:
from algorithms import *
from funcs import *
import numpy as np

In [ ]:
# q1, q2, f4
# 3 LRS (2 типа h, разные гиперпараметры), Dichotomy, Armijo * 2 (x_0)

func = BiFuncStatsDecorator(fopp3)
x_0 = np.array([1.2, 1.2])
PLOT_SIZE=3

In [ ]:
import matplotlib.pyplot as plt


def plot_trajectory(func: BiFunc, trajectory: np.ndarray, title=None):
    # Create a meshgrid for the 3D plot
    x = np.linspace(-PLOT_SIZE, PLOT_SIZE, 100)
    y = np.linspace(-PLOT_SIZE, PLOT_SIZE, 100)
    X, Y = np.meshgrid(x, y)
    Z = np.zeros_like(X)

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i, j] = func(np.array([X[i, j], Y[i, j]]))

    # Plot the 3D surface
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.6)  # type: ignore

    # Plot the trajectory
    ax.plot(trajectory[:, 0], trajectory[:, 1], [func(np.array([x, y]))
            for x, y in trajectory], color='r', marker='o')

    ax.scatter(trajectory[-1, 0], trajectory[-1, 1], func(trajectory[-1]), color='b', label='Final point')

    if title:
        ax.set_title(title)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')  # type: ignore
    #ax.set_zlim(-2, 20)
    plt.show()

In [ ]:
from typing import Dict

true_minima: Dict[BiFunc, float] = {
    q1: -1.9,
    q2: 0.0,
    f4: -0.514753641275705599276576050856482912233727798097409,
    f5: -0.5,
    f6: 0,
    fopp: -657.573,
    fopp2: 0,
    fopp3: -0.119789,
    fsinsin: -2,
}

assert func.f in true_minima

def print_stats(func: BiFuncStatsDecorator, trajectory: np.ndarray, title=None):

    cc, gc, hc = func.call_count, func.gradient_count, func.hessian_count
    calculated_min = func(trajectory[-1])
    print("\ntitle: ", title)
    print(f'Iterations: {len(trajectory) - 1}')
    print(f'x: {trajectory[-1]} f(x): {calculated_min}')
    print(f'Function evaluations: {cc}')
    print(f'Gradient evaluations: {gc}')
    print(f'Hessian evaluations: {hc}')
    print(f'True minimum: {true_minima[func.f]}')
    print(f'Error: {abs(calculated_min - true_minima[func.f])}')
    #plot_trajectory(func, trajectory, title)
    func.reset()

In [ ]:
from scipy.optimize import fmin_cg
from collections import namedtuple

xs = [[0.0, 0.0], [1.0, 4.0], [1.0, 1.0]]
xs = [np.array(x) for x in xs]

h_map = {
    constant_h(0.1): "constant_h(0.1)",
    exponential_decay(0.5): "exponential_decay(0.5)",
    exponential_decay(0.3): "exponential_decay(0.3)"
}

h_array = h_map.keys()
h_name_array = list(h_map.values())
func_array = [BiFuncStatsDecorator(q1), BiFuncStatsDecorator(q2), BiFuncStatsDecorator(f4)]

algorithms = {
    "Learning rate scheduling": {
        "algorithm": learning_rate_scheduling,
        "applier": lambda args, h: learning_rate_scheduling(args['x_0'], args['func'], h, args['stop_condition'])
    },
    "Armijo Gradient Descent": {
        "algorithm": steepest_gradient_descent_armijo,
        "applier": lambda args: steepest_gradient_descent_armijo(args['x_0'], args['func'], args['stop_condition'])
    },
    "Dichotomy Gradient Descent": {
        "algorithm": steepest_gradient_descent_dichotomy,
        "applier": lambda args: steepest_gradient_descent_dichotomy(args['x_0'], args['func'], args['eps'], args['stop_condition'])
    },
    "Scipy Wolfe Gradient Descent": {
        "algorithm": steepest_gradient_descent_scipy_wolfe,
        "applier": lambda args: steepest_gradient_descent_scipy_wolfe(args['x_0'], args['func'], args['stop_condition'])
    },
    "Newton Descent with 1D Search": {
        "algorithm": newton_descent_with_1d_search,
        "applier": lambda args: newton_descent_with_1d_search(args['x_0'], args['func'], args['stop_condition'], armijo_step_selector)
    },
    "Dog Leg": {
        "algorithm": damped_newton_descent,
        "applier": lambda args: damped_newton_descent(args['x_0'], args['func'], args['stop_condition'], constant_h(0.1))
    }
}

Stat = namedtuple('Stat', 'func_calls grad_calls trajectory')

def get_stat(func, trajectory) -> Stat:
    stat: Stat = Stat(func.call_count, func.gradient_count, trajectory)
    func.reset()
    return stat

eps = 1e-9
stat_array = []

for x_0 in xs:
    args = {
        "x_0": x_0,
        "func": func,
        "stop_condition": relative_x_condition(),
        "eps": eps
    }
    stats = [[get_stat(func, algorithms["Learning rate scheduling"]["applier"](args, h)) for h in h_array]]
    algorithms_values = list(algorithms.values())[1:]
    for algorithm_info in algorithms_values:
        stats.append([get_stat(func, algorithm_info["applier"](args))])
    stat_array.append((x_0, func, stats))
        
alg_name_array = list(algorithms.keys())

def print_stat(index: int):
    h = ""
    for stat in stat_array:
        x_0 = stat[0]
        func: BiFuncStatsDecorator = stat[1]
        inner_stat_array = stat[2][index]
        for stat_index in range(len(inner_stat_array)):
            if (index == 0):
                h = h_name_array[stat_index]
            func.call_count = inner_stat_array[stat_index].func_calls
            func.gradient_count = inner_stat_array[stat_index].grad_calls
            print_stats(func, inner_stat_array[stat_index].trajectory, alg_name_array[index] + f' x0={x_0} {h}')
    print("-------------------------------------------------------------------")
    
# Выводим ответ    
for i in range(len(stat_array[0][2])):
    print_stat(i)

In [ ]:

h = polynomial_decay(0.5, 1)
h = geometric_h()
h = constant_h(0.01)
#h = exponential_decay(0.1)

trajectory = learning_rate_scheduling(x_0, func, h, relative_x_condition())
print_stats(func, trajectory, "Learning rate scheduling")


In [ ]:
eps = 1e-9
trajectory = steepest_gradient_descent_dichotomy(
    x_0, func, eps, relative_x_condition())
print_stats(func, trajectory, "Dichotomy Gradient Descent")

In [ ]:
trajectory = steepest_gradient_descent_armijo(x_0, func, relative_x_condition())
print_stats(func, trajectory, "Armijo Gradient Descent")

In [ ]:
trajectory = steepest_gradient_descent_wolfe(x_0, func, relative_x_condition())
print_stats(func, trajectory, "Wolfe Gradient Descent")

In [ ]:
trajectory = steepest_gradient_descent_scipy_wolfe(x_0, func, relative_x_condition())
print_stats(func, trajectory, "Scipy Wolfe Gradient Descent")

In [ ]:
trajectory = damped_newton_descent(x_0, func, relative_x_condition(), constant_h(0.1))
print_stats(func, trajectory, "Damped Newton Method")

In [ ]:
def armijo_step_selector(k, x, grad, func):
    return armijo(x, func, grad)
trajectory = newton_descent_with_1d_search(x_0, func, relative_x_condition(), armijo_step_selector)
print_stats(func, trajectory, "Newton method with 1d search (armijo)")

In [ ]:
trajectory = bfgs(x_0, func, 1e-3)
print_stats(func, trajectory, "BFGS")

In [ ]:
from scipy.optimize import fmin_cg

# Conjugate Gradient Descent
fmin_cg(
    func,
    x_0,
    func.gradient,
    disp=True
)